In [ ]:
<h3>Importing necessary packages for notebook.</h3>

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

<h3>Make url call, get response, and parse the response contents using Beautiful Soup.</h3>

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
results_page = BeautifulSoup(response.content, "html.parser")

<h3>Extract the table head and table rows from the results.</h3>

In [ ]:
table = results_page.find("table")
table_items = table.find_all("tr")
table_head = table_items[0].find_all("th")
table_rows = table_items[1:]

<h3>Create the dataframe with it being initialized as empty with requisite column names coming from table head.</h3>

In [ ]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
toronto_neighborhoods = pd.DataFrame(columns = column_names)

<h3>Loop over the rows in table rows and append the applicable data in the dataframe. Ignore data where Borough is not assigned and assign Neighborhood to Borough if Borough is assigned while Neighborhood is not. If more than one neighborhood is assigned to the same postal code include all neighborhoods in same neighborhood column of dataframe for applicable postal code.</h3>

In [ ]:
for row in table_rows:
    
    row = row.find_all("td")
    PostalCode = row[0].get_text()
    Borough = row[1].get_text()
    Neighborhood = row[2].get_text().strip("\n")
    
    if Neighborhood == "Not assigned" and Borough != "Not assigned":
        Neighborhood = Borough
    
    if Borough == "Not assigned":
        continue
    else:
        if PostalCode in toronto_neighborhoods["PostalCode"].tolist():
            index = toronto_neighborhoods["PostalCode"].tolist().index(PostalCode)
            toronto_neighborhoods.loc[[index], ["Neighborhood"]] = toronto_neighborhoods.loc[[index], ["Neighborhood"]] + ", " + Neighborhood 
        else:
            toronto_neighborhoods = toronto_neighborhoods.append({
                "PostalCode": PostalCode,
                "Borough": Borough,
                "Neighborhood": Neighborhood
            }, ignore_index = True)

<h3>See the dataframe and its contents.</h3>

In [ ]:
toronto_neighborhoods

<h3>See what shape the datafame is.</h3>

In [ ]:
toronto_neighborhoods.shape

<h3>Read in a csv file containg latitude and longitude coordinates for the applicable toronto postal codes and store the data in a dataframe. Then join the two daataframes so all data is in one dataframe.</h3>

In [ ]:
lat_long_df = pd.read_csv("./Geospatial_Coordinates.csv")
toronto_neigh_with_coord = toronto_neighborhoods.join(lat_long_df.set_index(["Postal Code"]), on = "PostalCode")
toronto_neigh_with_coord